If you want to get something done you should start from the start. This is Day 1 of orientation microscopy. We create a fake pattern. We create a library that contains the fake pattern, and some other patterns. We check the process works as expected. We now step out of the calibration comfort zone!

In [1]:
#plt.close('all')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib tk
import numpy as np
import pyxem as pxm
from matplotlib import pyplot as plt
import hyperspy.api as hs

In [4]:
from pyxem.diffraction_generator import DiffractionSimulation
from pyxem.indexation_generator import IndexationGenerator
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks

In [5]:
dps = []
for alpha in [0,1,2,3]:
    dp_raw = np.ones((144,144))/5
    dp_raw[90:110,90:110] = 0.5
    dps.append(dp_raw)

dp = pxm.ElectronDiffraction([dps[0:2],dps[2:]])
#dp.plot(cmap='viridis')
#plt.show()
print(np.min(dp.data))

0.2


In [6]:
### Build a fake library:

library = dict()
library["Phase"] = {}
for alpha in np.arange(0,20,1):
    rotation = (alpha,0,0)
    local_cords = np.random.rand(25,2)*40
    library["Phase"][rotation] = DiffractionSimulation(coordinates=local_cords,intensities=np.ones_like(local_cords[:,0]))

In [7]:
dp

<ElectronDiffraction, title: , dimensions: (2, 2|144, 144)>

In [8]:
indexer = IndexationGenerator(dp,library)

In [9]:
from pyxem.utils import correlate
correlate(dp.inav[1,1].data,library["Phase"][(1,0,0)])

0.0041518138791161981

In [10]:
match_results = indexer.correlate(n_largest=20)

A Jupyter Widget

In [11]:
match_results.data[0][0][0:2]

array([[  0.00000000e+00,   1.60000000e+01,   0.00000000e+00,
          0.00000000e+00,   4.82508099e-03],
       [  0.00000000e+00,   6.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   4.48844744e-03]])

In [12]:
peaks= match_results.map(peaks_from_best_template,phase=["Phase"],library=library,inplace=False)

A Jupyter Widget

In [13]:
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
dp.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx+72,y=my+72,color='red',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=True)